### Pre-processing for DBT challenge 2:



TODO:
- Save greyscale of the slice
- Save color of RGB (previous, current next) slices.



In [5]:
import os

%matplotlib inline
import matplotlib.pyplot as plt

from duke_dbt_data import dcmread_image, read_boxes, draw_box

# imports 
import omidb
import numpy as np
import cv2
import csv, time


In [6]:
# base_path = "/mnt/mia_images/breast/DBT-Challenge"
# base_path = '/mnt/mia_images/breast/DBTex2/manifest-1617905855234/Breast-Cancer-Screening-DBT'
base_path = '/mnt/mia_images/breast/DBTex2'
# base_path = "/home/robert/data/DBT2"

# df = read_boxes(boxes_fp=base_path+"/rob_BCS-DBT boxes-local_eval.csv", filepaths_fp=base_path+"/rob_BCS-DBT file-paths-local_eval.csv")
df = read_boxes(boxes_fp=base_path+"/BCS-DBT boxes-train-v2.csv", filepaths_fp=base_path+"/BCS-DBT file-paths-train PHASE 2.csv")


debug = False

In [7]:
print(len(df))

224


In [8]:
print(df)

      PatientID    StudyUID  View  Subject  Slice     X     Y  Width  Height  \
0    DBT-P00013  DBT-S00163  rmlo        0     16  1116  1724    218     105   
1    DBT-P00024  DBT-S03255   lcc        0     19   267   488     64      64   
2    DBT-P00024  DBT-S03255   lcc        0     37   169   384     93      85   
3    DBT-P00024  DBT-S03255  lmlo        0     11   471  1060     67      56   
4    DBT-P00024  DBT-S03255  lmlo        0     12   640   673     93      80   
..          ...         ...   ...      ...    ...   ...   ...    ...     ...   
219  DBT-P05030  DBT-S05569  rmlo        2     24  1465   624     84     115   
220  DBT-P05047  DBT-S05588   rcc        0     41  1341  1073    387     499   
221  DBT-P05047  DBT-S05588  rmlo        0     41  1192  1019    456     541   
222  DBT-P05056  DBT-S01839   rcc        0     44  1642   661    245     392   
223  DBT-P05056  DBT-S01839  rmlo        0     22  1167   499    439     315   

      Class  AD  VolumeSlices  \
0    b

In [9]:
output_path = '/home/robert/data/DBT2'
output_folder = '/train'
# create folders
try:
    os.mkdir(output_path)     
except OSError as e:
    if (e.errno != 17): 
        print("copy_case: creation of %s failed" % output_path)
        print(e)
try: 
    os.mkdir(output_path+output_folder)
#    os.mkdir(output_path+output_folder) todo: save lesion patches. 
    
except OSError as e:
    if (e.errno != 17): 
        print("copy_case: creation of %s failed" % output_path)
        print(e)


In [24]:
def get_normal_BBox (image,bbox):
    #threshold image 
    img = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY)[1]  # ensure binary
#     plt.imshow(img,'gray')#,vmin=0,vmax=255))
#     plt.show()
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=4)
    sizes = stats[:, -1]
    max_label = 1
    max_size = sizes[1]
    for i in range(2, nb_components):
        if sizes[i] > max_size:
            max_label = i
            max_size = sizes[i]
    img2 = np.zeros(output.shape,dtype=np.uint8)
    img2[output == max_label] = 255
#     plt.imshow(img2,'gray')
#     plt.show()
#     input("Press Enter to continue...")
    contours, hierarchy = cv2.findContours(img2,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

    cnt = contours[0]
    aux_im = img2
    x,y,w,h = cv2.boundingRect(cnt)
    if (debug):    
        cv2.rectangle(aux_im,(x,y),(x+w,y+h),(255,0,0),5)
        plt.imshow(aux_im)
        plt.show()
    #input("Press Enter to continue...")
    out_bbox = omidb.mark.BoundingBox(x, y, x+w, y+h)
    
    return out_bbox, img2 # returns bounding box and mask image. 

In [9]:
# def copy_case(view, client, episode,image,side, bbox_roi, slice_index):
    
#     filename =client+"_"+episode+"_"+view+"_"+str(slice_index)+".png"
#     print(filename)
# #   Assuming windowWidth has been already applied (see Duke code)    
# #     if 'WindowWidth' in image.dcm:
# #         #print('Image hs windowing')
# #         image_2d = apply_voi_lut(image.dcm.pixel_array, image.dcm)
# #     else: image_2d = image.dcm.pixel_array
#     # Rescaling grey scale between 0-255
   
#     dims = image.shape
#     image_2d_scaled = (np.maximum(image,0) / image.max()) * 255.0

#     # Convert to uint
#     image_2d_scaled = np.uint8(image_2d_scaled)
#     if (side=='r' or side=='R'): # flip image and ROI coordinates.
#         image_2d_scaled =cv2.flip(image_2d_scaled, 1)
#         aux = bbox_roi.x2 
#         bbox_roi.x2 = dims[1]-bbox_roi.x1 
#         bbox_roi.x1 = dims[1]-aux 
#         print("bbox_roi",bbox_roi)

#     dims = image_2d_scaled.shape
#     print("dimension",dims)
   
#     # get ROI from mask
#     # Write the PNG file
# #     aux_folder = output_path+output_folder+"/"+ filename
#     #adding an extra_size around the lesion. TODO: save lesion patches. 
# #     y1 = np.maximum(bbox_roi.y1-extra_size,0)
# #     y2 = np.minimum(bbox_roi.y2+extra_size,dims[0])
# #     x1 = np.maximum(bbox_roi.x1-extra_size,0)
# #     x2 = np.minimum(bbox_roi.x2+extra_size,dims[1])
# #         image_crop = image_2d_scaled[y1:y2,x1:x2]
# #         cv2.imwrite(aux_folder,image_crop)

#     aux_folder = output_path+output_folder+"/"+ filename
#     bbox,mask = get_normal_BBox(image_2d_scaled,bbox_roi)
#     print(bbox)
#     # TODO: get random patches with get_random_bbox(bbox)
#     #bbox_norm = get_random_bbox(bbox, bbox_roi,mask)
# #     image_crop = image_2d_scaled[bbox_norm.y1:bbox_norm.y2,bbox_norm.x1:bbox_norm.x2]
# #     cv2.imwrite(aux_folder,image_crop)

#     print(aux_folder)        
#     # Write the PNG file
#     image_crop = image_2d_scaled[bbox.y1:bbox.y2,bbox.x1:bbox.x2]
#     cv2.imwrite(aux_folder,image_crop)
    
#     # adapt bbox_ROI to new cropped image (remove bbox.x1 and bbox.y1)
#     bbox_roi.x1 = bbox_roi.x1 - bbox.x1
#     bbox_roi.y1 = bbox_roi.y1 - bbox.y1
#     bbox_roi.x2 = bbox_roi.x2 - bbox.x1
#     bbox_roi.y2 = bbox_roi.y2 - bbox.y1
    
#     if (debug):       
#         cv2.rectangle(image_crop,(bbox_roi.x1,bbox_roi.y1),(bbox_roi.x2,bbox_roi.y2),(255,0,0),5)
#         plt.imshow(image_crop)
#         plt.show()
    
#     # write CSV 
#     with open(output_path+'/train_bboxes.csv', 'a+', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow([client, episode, filename, side, bbox,  slice_index, bbox_roi,])
        
    
               

In [52]:
def copy_case_rgb(view, client, episode,image_rgb,side, bbox_roi, slice_index):
    
    filename =client+"_"+episode+"_"+view+"_"+str(slice_index)+"_rgb.png"
    print(filename)
    
    print(image_rgb.shape)
    
    if (image_rgb.ndim>2): # contains rgb image
        comp = image_rgb.shape[0]
    else: comp = 1
    
    for nc in range(comp):
        
        if comp ==1: image = image_rgb # image of only component.
        else: image =  image_rgb[nc]
            
        dims = image.shape
        image_2d_scaled = (np.maximum(image,0) / image.max()) * 255.0

        # Convert to uint
        image_2d_scaled = np.uint8(image_2d_scaled)
        if (side=='r' or side=='R'): # flip image and ROI coordinates.
            image_2d_scaled =cv2.flip(image_2d_scaled, 1)
            aux = bbox_roi.x2 
            bbox_roi.x2 = dims[1]-bbox_roi.x1 
            bbox_roi.x1 = dims[1]-aux 
            print("bbox_roi",bbox_roi)

        dims = image_2d_scaled.shape
        print("dimension",dims)  
        
        if (nc == 0) : # same bbox for all slices
            bbox,mask = get_normal_BBox(image_2d_scaled,bbox_roi)
            print(bbox)           
        image_crop = image_2d_scaled[bbox.y1:bbox.y2,bbox.x1:bbox.x2]
        print(image_crop.shape)
        if (nc==0) and (comp>1): 
                out_rgb = np.zeros([image_crop.shape[0],image_crop.shape[1],comp])
        if (comp==1): out_rgb = image_crop
        else: out_rgb[:,:,nc] = image_crop
        
    aux_folder = output_path+output_folder+"/"+ filename
    print(aux_folder)        
    cv2.imwrite(aux_folder,out_rgb)
    
    # adapt bbox_ROI to new cropped image (remove bbox.x1 and bbox.y1)
    bbox_roi.x1 = bbox_roi.x1 - bbox.x1
    bbox_roi.y1 = bbox_roi.y1 - bbox.y1
    bbox_roi.x2 = bbox_roi.x2 - bbox.x1
    bbox_roi.y2 = bbox_roi.y2 - bbox.y1
    
    if (debug):       
        cv2.rectangle(image_crop,(bbox_roi.x1,bbox_roi.y1),(bbox_roi.x2,bbox_roi.y2),(255,0,0),5)
        plt.imshow(image_crop)
        plt.show()
    
    # write CSV 
    with open(output_path+'/train_bboxes.csv', 'a+', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([client, episode, filename, side, bbox,  slice_index, bbox_roi,])
        
    
               

## Generate training images

In [53]:
for index,box_series in df.iterrows():
    start_time = time.time()
    #box_series = df.iloc[0]
    print(box_series["PatientID"],box_series["View"],box_series["Slice"], box_series["Class"])
    view = box_series["View"]
    slice_index = box_series["Slice"]
    # if you have image data saved in classic path folder structure, read the file path from "classic_path"
    # image_path = os.path.join("/data", view_series["classic_path"])
    image_path = os.path.join(base_path, box_series["descriptive_path"])
    # %%time
    
#     image = dcmread_image(fp=image_path, view=view, index=slice_index)
    image3D = dcmread_image(fp=image_path, view=view)
    l_time = time.time()
    print("loading t", l_time-start_time)
        
    image = image3D[slice_index]
    
    image_org = np.copy(image)
    x, y, w, h = box_series[["X", "Y", "Width", "Height"]]
    bbox = omidb.mark.BoundingBox(x, y, x+w, y+h)
    client = box_series["PatientID"]
    episode = box_series["StudyUID"]
    side = view[0]
    # copy grey-scale image
    image_rgb = []
    image_rgb.append(image3D[slice_index-2])
    image_rgb.append(image3D[slice_index])
    image_rgb.append(image3D[slice_index+2])
    image_rgb = np.array(image_rgb)
    print (image_rgb.shape)
    copy_case_rgb(view,client , episode,np.array(image_rgb),side, bbox, slice_index)
#     copy_case_rgb(view,client , episode,image3D[slice_index],side, bbox, slice_index) version greyscale
    p_time = time.time()
    print("processingTime",p_time-l_time)
#     input("copy 1 case")
    
    # copy color image
    
    if (debug):
        image_org = draw_box(image=image_org, x=x, y=y, width=w, height=h, lw=10)    
        plt.imshow(image_org, cmap=plt.cm.gray);
        plt.show()

DBT-P00013 rmlo 16 benign
loading t 22.80126714706421
<class 'numpy.ndarray'>
(3, 2457, 1890)
DBT-P00013_DBT-S00163_rmlo_16_rgb.png
(3, 2457, 1890)
bbox_roi BoundingBox(x1=556, y1=1724, x2=774, y2=1829)
dimension (2457, 1890)
BoundingBox(x1=0, y1=0, x2=954, y2=2457)
(2457, 954)
bbox_roi BoundingBox(x1=1116, y1=1724, x2=1334, y2=1829)
dimension (2457, 1890)
(2457, 954)
bbox_roi BoundingBox(x1=556, y1=1724, x2=774, y2=1829)
dimension (2457, 1890)
(2457, 954)
/home/robert/data/DBT2/train/DBT-P00013_DBT-S00163_rmlo_16_rgb.png
processingTime 0.3087174892425537
DBT-P00024 lcc 19 benign
loading t 39.06942820549011
<class 'numpy.ndarray'>
(3, 2457, 1996)
DBT-P00024_DBT-S03255_lcc_19_rgb.png
(3, 2457, 1996)
dimension (2457, 1996)
BoundingBox(x1=0, y1=140, x2=1052, y2=2225)
(2085, 1052)
dimension (2457, 1996)
(2085, 1052)
dimension (2457, 1996)
(2085, 1052)
/home/robert/data/DBT2/train/DBT-P00024_DBT-S03255_lcc_19_rgb.png
processingTime 0.3006589412689209
DBT-P00024 lcc 37 benign
loading t 38.38

loading t 20.376075983047485
<class 'numpy.ndarray'>
(3, 2457, 1996)
DBT-P00361_DBT-S00216_rcc_17_rgb.png
(3, 2457, 1996)
bbox_roi BoundingBox(x1=346, y1=1041, x2=600, y2=1308)
dimension (2457, 1996)
BoundingBox(x1=0, y1=237, x2=928, y2=2054)
(1817, 928)
bbox_roi BoundingBox(x1=1396, y1=1041, x2=1650, y2=1308)
dimension (2457, 1996)
(1817, 928)
bbox_roi BoundingBox(x1=346, y1=1041, x2=600, y2=1308)
dimension (2457, 1996)
(1817, 928)
/home/robert/data/DBT2/train/DBT-P00361_DBT-S00216_rcc_17_rgb.png
processingTime 0.32172107696533203
DBT-P00361 rmlo 25 benign
loading t 23.05852198600769
<class 'numpy.ndarray'>
(3, 2457, 1996)
DBT-P00361_DBT-S00216_rmlo_25_rgb.png
(3, 2457, 1996)
bbox_roi BoundingBox(x1=380, y1=1011, x2=603, y2=1195)
dimension (2457, 1996)
BoundingBox(x1=0, y1=0, x2=876, y2=1866)
(1866, 876)
bbox_roi BoundingBox(x1=1393, y1=1011, x2=1616, y2=1195)
dimension (2457, 1996)
(1866, 876)
bbox_roi BoundingBox(x1=380, y1=1011, x2=603, y2=1195)
dimension (2457, 1996)
(1866, 876)
/

bbox_roi BoundingBox(x1=1163, y1=1242, x2=1396, y2=1443)
dimension (2457, 1996)
(2226, 1460)
/home/robert/data/DBT2/train/DBT-P00818_DBT-S02315_rcc_44_rgb.png
processingTime 0.4278738498687744
DBT-P00818 rmlo 42 benign
loading t 61.842519760131836
<class 'numpy.ndarray'>
(3, 2457, 1996)
DBT-P00818_DBT-S02315_rmlo_42_rgb.png
(3, 2457, 1996)
bbox_roi BoundingBox(x1=1200, y1=1335, x2=1425, y2=1557)
dimension (2457, 1996)
BoundingBox(x1=0, y1=0, x2=1498, y2=2457)
(2457, 1498)
bbox_roi BoundingBox(x1=571, y1=1335, x2=796, y2=1557)
dimension (2457, 1996)
(2457, 1498)
bbox_roi BoundingBox(x1=1200, y1=1335, x2=1425, y2=1557)
dimension (2457, 1996)
(2457, 1498)
/home/robert/data/DBT2/train/DBT-P00818_DBT-S02315_rmlo_42_rgb.png
processingTime 0.4773445129394531
DBT-P00827 rcc 33 benign
loading t 20.29325771331787
<class 'numpy.ndarray'>
(3, 2457, 1890)
DBT-P00827_DBT-S01731_rcc_33_rgb.png
(3, 2457, 1890)
bbox_roi BoundingBox(x1=219, y1=1725, x2=363, y2=1872)
dimension (2457, 1890)
BoundingBox(x1

bbox_roi BoundingBox(x1=14, y1=858, x2=646, y2=1456)
dimension (2457, 1890)
(2457, 852)
/home/robert/data/DBT2/train/DBT-P01262_DBT-S00537_rmlo_27_rgb.png
processingTime 0.35433435440063477
DBT-P01267 rcc 46 cancer
loading t 21.207541465759277
<class 'numpy.ndarray'>
(3, 2457, 1890)
DBT-P01267_DBT-S03415_rcc_46_rgb.png
(3, 2457, 1890)
bbox_roi BoundingBox(x1=1, y1=1123, x2=186, y2=1444)
dimension (2457, 1890)
BoundingBox(x1=0, y1=553, x2=624, y2=2113)
(1560, 624)
bbox_roi BoundingBox(x1=1704, y1=1123, x2=1889, y2=1444)
dimension (2457, 1890)
(1560, 624)
bbox_roi BoundingBox(x1=1, y1=1123, x2=186, y2=1444)
dimension (2457, 1890)
(1560, 624)
/home/robert/data/DBT2/train/DBT-P01267_DBT-S03415_rcc_46_rgb.png
processingTime 0.2772483825683594
DBT-P01267 rmlo 41 cancer
loading t 24.603552103042603
<class 'numpy.ndarray'>
(3, 2457, 1890)
DBT-P01267_DBT-S03415_rmlo_41_rgb.png
(3, 2457, 1890)
bbox_roi BoundingBox(x1=91, y1=796, x2=444, y2=1213)
dimension (2457, 1890)
BoundingBox(x1=0, y1=0, x2=

bbox_roi BoundingBox(x1=418, y1=839, x2=548, y2=955)
dimension (2457, 1890)
(2050, 942)
/home/robert/data/DBT2/train/DBT-P01587_DBT-S04115_rcc_34_rgb.png
processingTime 0.31671762466430664
DBT-P01587 rmlo 17 benign
loading t 31.324328422546387
<class 'numpy.ndarray'>
(3, 2457, 1890)
DBT-P01587_DBT-S04115_rmlo_17_rgb.png
(3, 2457, 1890)
bbox_roi BoundingBox(x1=579, y1=1048, x2=706, y2=1152)
dimension (2457, 1890)
BoundingBox(x1=0, y1=0, x2=980, y2=2457)
(2457, 980)
bbox_roi BoundingBox(x1=1184, y1=1048, x2=1311, y2=1152)
dimension (2457, 1890)
(2457, 980)
bbox_roi BoundingBox(x1=579, y1=1048, x2=706, y2=1152)
dimension (2457, 1890)
(2457, 980)
/home/robert/data/DBT2/train/DBT-P01587_DBT-S04115_rmlo_17_rgb.png
processingTime 0.32098865509033203
DBT-P01624 lcc 20 cancer
loading t 33.09942674636841
<class 'numpy.ndarray'>
(3, 2457, 1996)
DBT-P01624_DBT-S03518_lcc_20_rgb.png
(3, 2457, 1996)
dimension (2457, 1996)
BoundingBox(x1=0, y1=324, x2=1126, y2=1991)
(1667, 1126)
dimension (2457, 1996

...

loading t 39.20446538925171
<class 'numpy.ndarray'>
(3, 2457, 1996)
DBT-P02935_DBT-S00614_rcc_28_rgb.png
(3, 2457, 1996)
bbox_roi BoundingBox(x1=87, y1=991, x2=194, y2=1069)
dimension (2457, 1996)
BoundingBox(x1=0, y1=298, x2=1060, y2=2159)
(1861, 1060)
bbox_roi BoundingBox(x1=1802, y1=991, x2=1909, y2=1069)
dimension (2457, 1996)
(1861, 1060)
bbox_roi BoundingBox(x1=87, y1=991, x2=194, y2=1069)
dimension (2457, 1996)
(1861, 1060)
/home/robert/data/DBT2/train/DBT-P02935_DBT-S00614_rcc_28_rgb.png
processingTime 0.34717345237731934
DBT-P02935 rcc 35 cancer
loading t 38.545000314712524
<class 'numpy.ndarray'>
(3, 2457, 1996)
DBT-P02935_DBT-S00614_rcc_35_rgb.png
(3, 2457, 1996)
bbox_roi BoundingBox(x1=301, y1=1183, x2=487, y2=1353)
dimension (2457, 1996)
BoundingBox(x1=0, y1=294, x2=1060, y2=2158)
(1864, 1060)
bbox_roi BoundingBox(x1=1509, y1=1183, x2=1695, y2=1353)
dimension (2457, 1996)
(1864, 1060)
bbox_roi BoundingBox(x1=301, y1=1183, x2=487, y2=1353)
dimension (2457, 1996)
(1864, 1060

dimension (2457, 1890)
(2165, 1110)
/home/robert/data/DBT2/train/DBT-P03212_DBT-S02198_lcc_25_rgb.png
processingTime 0.36136746406555176
DBT-P03212 lmlo 11 benign
loading t 30.721096992492676
<class 'numpy.ndarray'>
(3, 2457, 1890)
DBT-P03212_DBT-S02198_lmlo_11_rgb.png
(3, 2457, 1890)
dimension (2457, 1890)
BoundingBox(x1=0, y1=0, x2=1167, y2=2457)
(2457, 1167)
dimension (2457, 1890)
(2457, 1167)
dimension (2457, 1890)
(2457, 1167)
/home/robert/data/DBT2/train/DBT-P03212_DBT-S02198_lmlo_11_rgb.png
processingTime 0.3592824935913086
DBT-P03218 lcc 51 benign
loading t 36.6587495803833
<class 'numpy.ndarray'>
(3, 2457, 1996)
DBT-P03218_DBT-S04050_lcc_51_rgb.png
(3, 2457, 1996)
dimension (2457, 1996)
BoundingBox(x1=0, y1=503, x2=969, y2=2263)
(1760, 969)
dimension (2457, 1996)
(1760, 969)
dimension (2457, 1996)
(1760, 969)
/home/robert/data/DBT2/train/DBT-P03218_DBT-S04050_lcc_51_rgb.png
processingTime 0.3213982582092285
DBT-P03218 lmlo 35 benign
loading t 50.03357815742493
<class 'numpy.nd

processingTime 0.31549525260925293
DBT-P03677 rcc 25 benign
loading t 23.37456202507019
<class 'numpy.ndarray'>
(3, 2457, 1890)
DBT-P03677_DBT-S00709_rcc_25_rgb.png
(3, 2457, 1890)
bbox_roi BoundingBox(x1=1, y1=1559, x2=471, y2=2034)
dimension (2457, 1890)
BoundingBox(x1=0, y1=365, x2=561, y2=2296)
(1931, 561)
bbox_roi BoundingBox(x1=1419, y1=1559, x2=1889, y2=2034)
dimension (2457, 1890)
(1931, 561)
bbox_roi BoundingBox(x1=1, y1=1559, x2=471, y2=2034)
dimension (2457, 1890)
(1931, 561)
/home/robert/data/DBT2/train/DBT-P03677_DBT-S00709_rcc_25_rgb.png
processingTime 0.2743842601776123
DBT-P03677 rcc 43 benign
loading t 24.936034440994263
<class 'numpy.ndarray'>
(3, 2457, 1890)
DBT-P03677_DBT-S00709_rcc_43_rgb.png
(3, 2457, 1890)
bbox_roi BoundingBox(x1=16, y1=1375, x2=337, y2=1763)
dimension (2457, 1890)
BoundingBox(x1=0, y1=336, x2=560, y2=2305)
(1969, 560)
bbox_roi BoundingBox(x1=1553, y1=1375, x2=1874, y2=1763)
dimension (2457, 1890)
(1969, 560)
bbox_roi BoundingBox(x1=16, y1=1375, 

bbox_roi BoundingBox(x1=233, y1=560, x2=473, y2=788)
dimension (2457, 1996)
(1836, 912)
/home/robert/data/DBT2/train/DBT-P04116_DBT-S03961_rcc_30_rgb.png
processingTime 0.3338639736175537
DBT-P04116 rmlo 18 benign
loading t 43.083816051483154
<class 'numpy.ndarray'>
(3, 2457, 1996)
DBT-P04116_DBT-S03961_rmlo_18_rgb.png
(3, 2457, 1996)
bbox_roi BoundingBox(x1=441, y1=1331, x2=725, y2=1576)
dimension (2457, 1996)
BoundingBox(x1=0, y1=0, x2=1012, y2=2242)
(2242, 1012)
bbox_roi BoundingBox(x1=1271, y1=1331, x2=1555, y2=1576)
dimension (2457, 1996)
(2242, 1012)
bbox_roi BoundingBox(x1=441, y1=1331, x2=725, y2=1576)
dimension (2457, 1996)
(2242, 1012)
/home/robert/data/DBT2/train/DBT-P04116_DBT-S03961_rmlo_18_rgb.png
processingTime 0.3723483085632324
DBT-P04326 lcc 12 benign
loading t 22.640069723129272
<class 'numpy.ndarray'>
(3, 2457, 1890)
DBT-P04326_DBT-S03750_lcc_12_rgb.png
(3, 2457, 1890)
dimension (2457, 1890)
BoundingBox(x1=0, y1=702, x2=546, y2=2105)
(1403, 546)
dimension (2457, 189

bbox_roi BoundingBox(x1=520, y1=1992, x2=632, y2=2082)
dimension (2457, 1890)
(2029, 1233)
/home/robert/data/DBT2/train/DBT-P04858_DBT-S04555_rcc_31_rgb.png
processingTime 0.41590189933776855
DBT-P04901 rcc 35 cancer
loading t 23.406635761260986
<class 'numpy.ndarray'>
(3, 2457, 1890)
DBT-P04901_DBT-S05032_rcc_35_rgb.png
(3, 2457, 1890)
bbox_roi BoundingBox(x1=30, y1=961, x2=356, y2=1336)
dimension (2457, 1890)
BoundingBox(x1=0, y1=10, x2=755, y2=2071)
(2061, 755)
bbox_roi BoundingBox(x1=1534, y1=961, x2=1860, y2=1336)
dimension (2457, 1890)
(2061, 755)
bbox_roi BoundingBox(x1=30, y1=961, x2=356, y2=1336)
dimension (2457, 1890)
(2061, 755)
/home/robert/data/DBT2/train/DBT-P04901_DBT-S05032_rcc_35_rgb.png
processingTime 0.36891818046569824
DBT-P04901 rmlo 35 cancer
loading t 31.84235382080078
<class 'numpy.ndarray'>
(3, 2457, 1890)
DBT-P04901_DBT-S05032_rmlo_35_rgb.png
(3, 2457, 1890)
bbox_roi BoundingBox(x1=189, y1=1064, x2=638, y2=1430)
dimension (2457, 1890)
BoundingBox(x1=0, y1=0, x

### Evaluation
Input: 
- csv of predictions
- csv of boxes gt
- csv of labels gt

using duke code. 



### Generate Boxes validation
Generates files for using the evaluation function

PatientID,StudyUID,View,Subject,Slice,X,Y,Width,Height,VolumeSlices

In [10]:
output_csv = "gt_local_evaluation.csv" 
with open(output_path+'/'+output_csv, 'w+', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['PatientID','StudyUID','View','Subject','Slice','X','Width','Y','Height','VolumeSlices'])
  
for index,box_series in df.iterrows():
    print(box_series)
    view = box_series["View"]
    slice_index = box_series["Slice"]
    subject = box_series["Subject"]
    # if you have image data saved in classic path folder structure, read the file path from "classic_path"
    # image_path = os.path.join("/data", view_series["classic_path"])
    image_path = os.path.join(base_path, box_series["descriptive_path"])
    # %%time
    
    image = dcmread_image(fp=image_path, view=view)
    dim = image.shape
    print("dim z", dim[0], "slice_index",slice_index)
    x, y, w, h = box_series[["X", "Y", "Width", "Height"]]
    client = box_series["PatientID"]
    episode = box_series["StudyUID"]
    side = view[0]
    depth = int(dim[0]/8)
    with open(output_path+'/'+output_csv, 'a+', newline='') as file:
        writer = csv.writer(file)
        print(client, episode, view, subject, int(slice_index), int(x),int(w), int(y), int(h), float(dim[0]))
        writer.writerow([client, episode, view, subject, int(slice_index), int(x),int(w), int(y), int(h), float(dim[0])])
    
    
    

PatientID                                                  DBT-P00013
StudyUID                                                   DBT-S00163
View                                                             rmlo
Subject                                                             0
Slice                                                              16
X                                                                1116
Y                                                                1724
Width                                                             218
Height                                                            105
Class                                                          benign
AD                                                                  0
VolumeSlices                                                       49
descriptive_path    Breast-Cancer-Screening-DBT/DBT-P00013/01-01-2...
classic_path        Breast-Cancer-Screening-DBT/DBT-P00013/1.2.826...
Name: 0, dtype: obje

/home/robert/miniconda2/envs/fast2/lib/python3.8/site-packages/openjpeg/utils.py:189: UserWarning: The (0028,0101) Bits Stored value '10' in the dataset does not match the component precision value '16' found in the JPEG 2000 data. It's recommended that you change the Bits Stored value to produce the correct output
  warnings.warn(


dim z 49 slice_index 16
DBT-P00013 DBT-S00163 rmlo 0 16 1116 218 1724 105 49.0
PatientID                                                  DBT-P00024
StudyUID                                                   DBT-S03255
View                                                              lcc
Subject                                                             0
Slice                                                              19
X                                                                 267
Y                                                                 488
Width                                                              64
Height                                                             64
Class                                                          benign
AD                                                                  0
VolumeSlices                                                       73
descriptive_path    Breast-Cancer-Screening-DBT/DBT-P00024/01-01-2...
classic_pat

dim z 80 slice_index 35
DBT-P00194 DBT-S00645 rmlo 0 35 1347 268 1730 255 80.0
PatientID                                                  DBT-P00225
StudyUID                                                   DBT-S02346
View                                                              lcc
Subject                                                             0
Slice                                                              54
X                                                                 266
Y                                                                1180
Width                                                             323
Height                                                            254
Class                                                          benign
AD                                                                  1
VolumeSlices                                                       86
descriptive_path    Breast-Cancer-Screening-DBT/DBT-P00225/01-01-2...
classic_pat

dim z 60 slice_index 36
DBT-P00303 DBT-S02436 rmlo 0 36 1341 253 764 279 60.0
PatientID                                                  DBT-P00361
StudyUID                                                   DBT-S00216
View                                                              rcc
Subject                                                             0
Slice                                                              17
X                                                                1396
Y                                                                1041
Width                                                             254
Height                                                            267
Class                                                          benign
AD                                                                  1
VolumeSlices                                                       50
descriptive_path    Breast-Cancer-Screening-DBT/DBT-P00361/01-01-2...
classic_path

...

dim z 70 slice_index 45
DBT-P03816 DBT-S03888 rcc 0 45 1806 178 1335 193 70.0
PatientID                                                  DBT-P03816
StudyUID                                                   DBT-S03888
View                                                             rmlo
Subject                                                             0
Slice                                                              58
X                                                                1353
Y                                                                 894
Width                                                             236
Height                                                            152
Class                                                          benign
AD                                                                  1
VolumeSlices                                                       85
descriptive_path    Breast-Cancer-Screening-DBT/DBT-P03816/01-01-2...
classic_path

dim z 80 slice_index 59
DBT-P04090 DBT-S01718 rmlo 2 59 1212 134 825 125 80.0
PatientID                                                  DBT-P04116
StudyUID                                                   DBT-S03961
View                                                              rcc
Subject                                                             0
Slice                                                              30
X                                                                1523
Y                                                                 560
Width                                                             240
Height                                                            228
Class                                                          benign
AD                                                                  1
VolumeSlices                                                       80
descriptive_path    Breast-Cancer-Screening-DBT/DBT-P04116/01-01-2...
classic_path

dim z 74 slice_index 34
DBT-P04429 DBT-S00568 lmlo 2 34 672 273 707 290 74.0
PatientID                                                  DBT-P04631
StudyUID                                                   DBT-S05515
View                                                              lcc
Subject                                                             2
Slice                                                               9
X                                                                 546
Y                                                                1059
Width                                                              84
Height                                                             69
Class                                                          cancer
AD                                                                  0
VolumeSlices                                                       70
descriptive_path    Breast-Cancer-Screening-DBT/DBT-P04631/01-01-2...
classic_path 

dim z 76 slice_index 27
DBT-P04818 DBT-S02975 rcc 2 27 1515 145 930 183 76.0
PatientID                                                  DBT-P04818
StudyUID                                                   DBT-S02975
View                                                             rmlo
Subject                                                             2
Slice                                                              26
X                                                                1510
Y                                                                1211
Width                                                             168
Height                                                            173
Class                                                          benign
AD                                                                  1
VolumeSlices                                                       79
descriptive_path    Breast-Cancer-Screening-DBT/DBT-P04818/01-01-2...
classic_path 

dim z 75 slice_index 30
DBT-P05022 DBT-S05195 rcc 0 30 1655 251 1460 176 75.0
PatientID                                                  DBT-P05022
StudyUID                                                   DBT-S05195
View                                                             rmlo
Subject                                                             0
Slice                                                              35
X                                                                1634
Y                                                                1271
Width                                                             236
Height                                                            207
Class                                                          benign
AD                                                                  0
VolumeSlices                                                       72
descriptive_path    Breast-Cancer-Screening-DBT/DBT-P05022/01-01-2...
classic_path